In [1]:
import time, sys, os, re, traceback
import logging
import ipynbname

# check if using jupyter notebook
def is_notebook():
    try:
        shell = get_ipython().__class__.__name__
    except:
        return False
    if shell == 'ZMQInteractiveShell':
        return True
    return False

def log_to_file(log_file, line_msg):
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)
    
    formatter = logging.Formatter('%(asctime)s:~:%(message)s')
    file_handler = logging.FileHandler(log_file)
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    
    logger.info(line_msg)
    logger.removeHandler(file_handler)
    
    return 1

def logtime(func, keep_log=True):
    def wrapper(*args, **kwargs):
        try:
            # check which funciton is running
            print(func.__name__)
            start = time.perf_counter()
            result = func(*args, **kwargs)
            # place holder for error name and error message
            errors = ('', '')
        except Exception as e:
            errrs = (type(e).__name__, str(e).splitlines()[0])
            raise
        finally:
            end = time.perf_counter()
            stack = traceback.extract_stack()
            filename, lineno, function_name, code = stack[-2]
            # look for arguments
            argsname = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
            time_use = round(end-start, 2) # in seconds
            print_msg = "{} (Line {}, *args: {}, **kwargs: {}), {} seconds".format(
                func.__name__, lineno, argsname, kwargs, time_use
            )
            print(print_msg)
            
            if keep_log:
                if is_notebook():
                    filename = ipynbname.name()
                msg = "{}:~:{}:~:{}:~:{}:~:{}:~:{}:~:{}:~:{}".format(
                    filename, lineno, func.__name__, 
                    argsname, kwargs, time_use, errors[0], errors[1]
                )
                log_dir = "runtime"
                log_file = "{}.log".format(func.__name__)
                log_path = "{}/{}".format(log_dir, log_file)
                if not os.path.exists(log_dir):            
                    os.mkdir(log_dir)                  # create log dir
                if not os.path.exists(log_path):
                    with open(log_path, 'a+') as f:    # create log file
                        print('New log file created!')
                log_to_file(log_path, msg)
                
        return result
    return wrapper

In [2]:
@logtime
def multiply(x, y):
    time.sleep(.1) # wait .1 seconds
    return x * y

for i in range(10): 
    multiply(i,i+1)

multiply
multiply (Line 7, *args: i,i+1, **kwargs: {}), 0.11 seconds
multiply
multiply (Line 7, *args: i,i+1, **kwargs: {}), 0.11 seconds
multiply
multiply (Line 7, *args: i,i+1, **kwargs: {}), 0.11 seconds
multiply
multiply (Line 7, *args: i,i+1, **kwargs: {}), 0.11 seconds
multiply
multiply (Line 7, *args: i,i+1, **kwargs: {}), 0.11 seconds
multiply
multiply (Line 7, *args: i,i+1, **kwargs: {}), 0.11 seconds
multiply
multiply (Line 7, *args: i,i+1, **kwargs: {}), 0.11 seconds
multiply
multiply (Line 7, *args: i,i+1, **kwargs: {}), 0.11 seconds
multiply
multiply (Line 7, *args: i,i+1, **kwargs: {}), 0.1 seconds
multiply
multiply (Line 7, *args: i,i+1, **kwargs: {}), 0.11 seconds


#### See runtime/multiply.log